# Skin Lesion Classifier - Approach 1

## Google Colab

In [ ]:
# Run this cell to mount Google Drive for Colab
from google.colab import drive
drive.mount('/content/drive/')
# !ls '/content/drive/My Drive/Colab Notebooks'

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/isic-2019')

In [ ]:
# !cp '/content/drive/My Drive/Colab Notebooks/ISIC_2019_Training_Input.zip' '/home/ISIC_2019_Training_Input.zip'
# !cp '/content/drive/My Drive/Colab Notebooks/ISIC_2019_Training_GroundTruth.csv' '/home/ISIC_2019_Training_GroundTruth.csv'
# !cp '/content/drive/My Drive/Colab Notebooks/ISIC_2019_Test_Input.zip' '/home/ISIC_2019_Test_Input.zip'
# !cp '/content/drive/My Drive/Colab Notebooks/Out_Distribution.zip' '/home/Out_Distribution.zip'
# !unzip -qq '/home/ISIC_2019_Training_Input.zip' -d '/home'
# !unzip -qq '/home/ISIC_2019_Test_Input.zip' -d '/home'
# !unzip -qq '/home/Out_Distribution.zip' -d '/home'

In [ ]:
# # Ref https://docs.fast.ai/performance.html
# !pip3 uninstall -y pillow pil jpeg libtiff libjpeg-turbo
# !CFLAGS="${CFLAGS} -mavx2" pip3 install --upgrade --no-cache-dir --force-reinstall --no-binary :all: --compile pillow-simd

## Environment

### Install Python Packages

In [ ]:
# !pip3 install -r requirements.txt

### Check whether you’re running Pillow or Pillow-SIMD?

In [ ]:
# According to the author, if PILLOW_VERSION has a postfix, it is Pillow-SIMD0.
# (Assuming that Pillow will never make a .postX release).
!python3 -c "from PIL import Image; print(Image.PILLOW_VERSION)"

### Whether Pillow or Pillow-SIMD is using libjpeg-turbo?

In [ ]:
from PIL import features, Image
from packaging import version

if version.parse(Image.PILLOW_VERSION) >= version.parse("5.4.0"):
    if features.check_feature('libjpeg_turbo'):
        print("libjpeg-turbo is on")
    else:
        print("libjpeg-turbo is not on")
else:
    print("libjpeg-turbo' status can't be derived - need Pillow(-SIMD)? >= 5.4.0 to tell, current version {}".format(Image.PILLOW_VERSION))

### Confirm TensorFlow can see the GPU

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print("Found GPU at: {}".format(device_name))

### System Information

In [ ]:
import tensorflow as tf
import platform
from tensorflow.python.client import device_lib
import keras

!python3 --version

print('\nKeras Version: ', keras.__version__)
print('\nTensorFlow Version: ', tf.VERSION)

print('\nNVIDIA:')
!nvcc --version
!nvidia-smi
!nvidia-smi topo -m

print('\nCPU:')
!lscpu

print('\nMemory:')
!cat /proc/meminfo

print('\nOS:')
print(platform.platform())

print('\nDevices:')
print(device_lib.list_local_devices())

## Common Parameters

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline

data_folder = 'C:\ISIC_2019'
# data_folder = '/home'
# data_folder = '/home/jupyter'
# data_folder = '/home/ubuntu'

model_folder = 'models'
history_folder = 'history'
pred_result_folder = 'predict_results'
out_dist_pred_result_folder = 'out_dist_predict_results'
    
workers = os.cpu_count()

# How to handle SVG fonts
plt.rcParams['svg.fonttype'] = 'none'

## Import Training Data

In [ ]:
from collections import Counter
from data import load_isic_training_data
from visuals import autolabel

training_image_folder = os.path.join(data_folder, 'ISIC_2019_Training_Input')
ground_truth_file = os.path.join(data_folder, 'ISIC_2019_Training_GroundTruth.csv')

df_ground_truth, known_category_names, unknown_category_name = load_isic_training_data(training_image_folder, ground_truth_file)
known_category_num = len(known_category_names)
print("Number of known categories: {}".format(known_category_num))
print(known_category_names, '\n')
unknown_category_num = 1
print("Number of unknown categories: {}".format(unknown_category_num))
print(unknown_category_name, '\n')
all_category_names = known_category_names + [unknown_category_name]
all_category_num = known_category_num + unknown_category_num

# mapping from category to index
print('Category to Index:')
category_to_index = dict((c, i) for i, c in enumerate(all_category_names))
print(category_to_index, '\n')

count_per_category = Counter(df_ground_truth['category'])
total_sample_count = sum(count_per_category.values())
print("Original training data has {} samples.".format(total_sample_count))
for i, c in enumerate(all_category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_per_category[i], count_per_category[i]*100/total_sample_count))

# Create a bar chart
fig, ax = plt.subplots(figsize=(8, 5))
fig.patch.set_facecolor('white')
# plt.bar(count_per_category.keys(), count_per_category.values())
rects = plt.bar(all_category_names, [count_per_category[i] for i in range(all_category_num)])
autolabel(ax, rects)
fig.tight_layout()

df_ground_truth.head()

### Shuffle and Split Original Training Data into Training  and Validation Sets

In [ ]:
from data import train_validation_split
from visuals import plot_grouped_2bars

df_train, df_val = train_validation_split(df_ground_truth)

# Training Set
sample_count_train = df_train.shape[0]
print("Training set has {} samples.".format(sample_count_train))
count_per_category_train = Counter(df_train['category'])
for i, c in enumerate(all_category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_per_category_train[i], count_per_category_train[i]*100/sample_count_train))

# Validation Set
sample_count_val = df_val.shape[0]
print("\nValidation set has {} samples.".format(sample_count_val))
count_per_category_val = Counter(df_val['category'])
for i, c in enumerate(all_category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_per_category_val[i], count_per_category_val[i]*100/sample_count_val))

plot_grouped_2bars(
    scalars=[[count_per_category_train[i] for i in range(all_category_num)],
             [count_per_category_val[i] for i in range(all_category_num)]],
    scalarlabels=['Training', 'Validation'],
    xticklabels=all_category_names,
    title='Distribution of Training and Validation Sets'
)

### Class Weights based on the Traning Set

In [ ]:
from data import compute_class_weight_dict

class_weight_dict, class_weights = compute_class_weight_dict(df_train)
print('Class Weights Dictionary (without UNK):')
print(class_weight_dict)

# Create a bar chart
fig, ax = plt.subplots(figsize=(7, 5))
fig.patch.set_facecolor('white')
ax.set_title('Class Weights')
plt.bar(known_category_names, [class_weight_dict[i] for i in range(known_category_num)]);

### Per-channel Mean and Standard Deviation over the Training Set

In [ ]:
from utils import calculate_mean_std

### Uncomment below codes to calculate per-channel mean and standard deviation over the training set
# rgb_mean, rgb_std = calculate_mean_std(df_train['path'])
# print("Mean:{}\nSTD:{}".format(rgb_mean, rgb_std))

# Output was:
# Mean:[0.6236094091893962, 0.5198354883713194, 0.5038435406338101]
# STD:[0.2421814437693499, 0.22354427793687906, 0.2314805420919389]

### Samples of each Known Category

In [ ]:
from IPython.display import Image

category_groups = df_train.groupby('category')

# Number of samples for each category
num_per_category = 3

fig, axes = plt.subplots(nrows=known_category_num, ncols=num_per_category, figsize=(9, 24))
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
fig.patch.set_facecolor('white')

for idx, val in enumerate(known_category_names):
    i = 0
    for index, row in category_groups.get_group(idx).head(num_per_category).iterrows():
        ax = axes[idx, i]
        ax.imshow(plt.imread(row['path']))
        ax.set_xlabel(row['image'])
        if ax.is_first_col():
            ax.set_ylabel(val, fontsize=20)
            ax.yaxis.label.set_color('blue')
        i += 1
    
fig.tight_layout()

## Create a Vanilla CNN as Benchmark Model

### Train the Vanilla CNN

In [ ]:
!python3 main.py /home --approach 1 --training --epoch 100 --batchsize 32 --maxqueuesize 10 --model Vanilla

### Model Complexity Graph of Vanilla CNN

In [ ]:
from visuals import *

fig = plot_complexity_graph(csv_file=os.path.join(history_folder, 'Vanilla.training.csv'),
                            title='Complexity Graph of Vanilla CNN')
fig.savefig(os.path.join(history_folder, 'Vanilla.training.svg'), format='svg')

### Classify Dermoscopic Images with the Vanilla CNN

In [ ]:
import random
from utils import path_to_tensor

def vanilla_classify(img_path, topk=5):
    # TODO: Use Image Augmentation Pipeline instead of path_to_tensor
    # Note that path_to_tensor does not rescale images like VanillaClassifier.preprocess_input
    predicted_vector = model.predict(path_to_tensor(img_path, size=(224, 224)))
    idx_topk = np.argsort(-predicted_vector)[0, :topk]
    probs = np.take(predicted_vector, idx_topk)
    names = [category_names[idx] for idx in idx_topk]
    
    return idx_topk, names, probs

topk = known_category_num
df_row = df_val.iloc[random.randrange(len(df_val.index))]
idx_topk, names, probs = vanilla_classify(df_row['path'], topk=topk)
# print(probs)

# Set up plot
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)
fig.patch.set_facecolor('white')

# Set up title
fig.suptitle(df_row['image'])

# Input Image
ax1.set_title(category_names[df_row['category']])
ax1.imshow(plt.imread(df_row['path']))

# Plot probabilities bar chart
ax2.set_title("Top {0} probabilities".format(topk))
ax2.barh(np.arange(topk), probs)
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(topk))
ax2.set_yticklabels(names, size='medium')
ax2.yaxis.tick_right()
ax2.set_xlim(0, 1.0)
ax2.invert_yaxis()

## Transfer Learning

### Train Models by Transfer Learning

In [ ]:
!python3 main.py /home --approach 1 --training --epoch 100 --batchsize 32 --maxqueuesize 10 --model DenseNet201 Xception ResNeXt50

### Complexity Graph of Transfer Learning Models

In [ ]:
from visuals import *

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
feature_extract_epochs = 3

for model_name in model_names:
    file_path = os.path.join(history_folder, "{}.training.csv".format(model_name))
    if os.path.exists(file_path):
        fig = plot_complexity_graph(csv_file=file_path,
                              title="Complexity Graph of {}".format(model_name),
                              figsize=(14, 10),
                              feature_extract_epochs=feature_extract_epochs)
        fig.savefig(os.path.join(history_folder, "{}.training.svg".format(model_name)), format='svg')

## Predict Validation Set

### Predict Validation Set by Different Models

In [ ]:
# !python3 main.py /home --approach 1 --predval --model Vanilla Xception DenseNet201 ResNeXt50
!python main.py C:\ISIC_2019 --approach 1 --predval --model Vanilla Xception DenseNet201 ResNeXt50

### Ensemble Models' Predictions on Validation Set

In [ ]:
from utils import ensemble_predictions

ensemble_predictions(pred_result_folder, known_category_names)

### Load Prediction Results on Validation Set

In [ ]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, recall_score
from visuals import plot_confusion_matrix
from keras.utils import np_utils
from keras_numpy_backend import categorical_crossentropy

model_names = ['Vanilla', 'DenseNet201', 'Xception', 'ResNeXt50', 'Ensemble']
postfix = 'best_balanced_acc'
print('Model selection criteria: ', postfix)

for model_name in model_names:
    # Load predicted results
    file_path = os.path.join(pred_result_folder, "{}_{}.csv".format(model_name, postfix))
    # file_path = os.path.join(pred_result_folder, "{}_best_loss.csv".format(model_name))
    if not os.path.exists(file_path):
        continue

    print("========== {} ==========".format(model_name))
    df = pd.read_csv(file_path)
    y_true = df['category']
    y_pred = df['pred_category']

    # Compute Balanced Accuracy
    print('balanced_accuracy_score: ', balanced_accuracy_score(y_true, y_pred))
    print('macro recall_score: ', recall_score(y_true, y_pred, average='macro'))

    # Compute categorical_crossentropy
    y_true_onehot = np_utils.to_categorical(df['category'], num_classes=known_category_num)
    y_pred_onehot = np.array(df.iloc[:,1:9])
    print('categorical_crossentropy: ',
          np.average(categorical_crossentropy(y_true_onehot, y_pred_onehot)))

    # Compute weighted categorical_crossentropy
    print('weighted categorical_crossentropy: ',
          np.average(categorical_crossentropy(y_true_onehot, y_pred_onehot, class_weights=class_weights)))

    # Confusion Matrix
    fig = plot_confusion_matrix(y_true, y_pred, known_category_names, normalize=True,
                                title="Confusion Matrix of {}".format(model_name),
                                figsize=(8, 6))
    print('')

In [ ]:
from visuals import plot_grouped_2bars

sample_count_val = y_true.shape[0]
print("Validation set has {} samples.\n".format(sample_count_val))

print('========== Ground Truth ==========')
count_true = Counter(y_true)
for i, c in enumerate(known_category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_true[i], count_true[i]*100/sample_count_val))

for model_name in model_names:
    # Load predicted results
    file_path = os.path.join(pred_result_folder, "{}_{}.csv".format(model_name, postfix))
    if not os.path.exists(file_path):
        continue

    print("\n========== {} Prediction ==========".format(model_name))
    df = pd.read_csv(file_path)
    y_pred = df['pred_category']
    
    count_pred = Counter(y_pred)
    for i, c in enumerate(known_category_names):
        print("'%s':\t%d\t(%.2f%%)" % (c, count_pred[i], count_pred[i]*100/sample_count_val))

    # Plot Prediction Distribution
    plot_grouped_2bars(
        scalars=[[count_true[i] for i in range(known_category_num)],
                 [count_pred[i] for i in range(known_category_num)]],
        scalarlabels=['Ground Truth', 'Prediction'],
        xticklabels=known_category_names,
        title="Prediction Distribution of {}".format(model_name)
    )

## Out-of-Distribution

### Create Out-of-Distribution CSV File

In [ ]:
from data import get_dataframe_from_img_folder

if not os.path.exists(out_dist_pred_result_folder):
    os.makedirs(out_dist_pred_result_folder)
    
out_dist_image_folder = os.path.join(data_folder, 'Out_Distribution')
out_dist_file = os.path.join(out_dist_pred_result_folder, 'Out_Distribution.csv')

df_out_dist = get_dataframe_from_img_folder(out_dist_image_folder, has_path_col=False)
print(len(df_out_dist), 'out-of-distribution images')
df_out_dist.to_csv(out_dist_file, index=False)

### Predict Out-of-Distribution Set by Different Models

In [ ]:
import pandas as pd
from keras.models import load_model
from metrics import balanced_accuracy
from lesion_classifier import LesionClassifier
from vanilla_classifier import VanillaClassifier
from base_model_param import get_transfer_model_param_map
from keras import backend as K

df_out_dist = pd.read_csv(out_dist_file)
df_out_dist['path'] = df_out_dist.apply(lambda row : os.path.join(out_dist_image_folder, row['image']+'.jpg'), axis=1)

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
postfixes = ['best_balanced_acc', 'best_loss', 'latest']
model_param_map = get_transfer_model_param_map()

for model_name in model_names:
    for postfix in postfixes:
        filename = "{}_{}.hdf5".format(model_name, postfix)
        print('Load: ', filename)
        # Load model
        model = load_model(filepath=os.path.join(model_folder, filename),
                           custom_objects={'balanced_accuracy': balanced_accuracy(known_category_num)})

        # Predict Out-of-Distribution Set
        df_softmax, df_logit = LesionClassifier.predict_dataframe(
            model=model, df=df_out_dist,
            category_names=known_category_names,
            augmentation_pipeline=LesionClassifier.create_aug_pipeline_val(model_param_map[model_name].input_size),
            preprocessing_function=model_param_map[model_name].preprocessing_func,
            workers=workers,
            softmax_save_file_name=os.path.join(out_dist_pred_result_folder, "{}_{}.csv".format(model_name, postfix)),
            logit_save_file_name=os.path.join(out_dist_pred_result_folder, "{}_{}_logit.csv".format(model_name, postfix)))

        del model
        K.clear_session()

### Ensemble Models' Predictions on Out-of-Distribution Set

In [ ]:
from utils import ensemble_predictions

ensemble_predictions(out_dist_pred_result_folder, known_category_names)

### Compute Baseline and ODIN Softmax Scores

In [ ]:
!python3 main.py /home --approach 1 --odinscore

### Tune ODIN Parameters

The optimal parameters are chosen to minimize the FPR at TPR 95%.

#### Baseline Softmax Scores

In [ ]:
from odin import ModelAttr, find_best_delta_at_tpr95, get_tpr_and_fpr, auroc
import sys
import numpy as np
from sklearn.model_selection import train_test_split

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
postfixes = ['best_balanced_acc', 'best_loss', 'latest']

fpr_min = sys.float_info.max
delta_fpr_min = None
modelattr_fpr_min = None

auroc_max = sys.float_info.min
odinparam_auroc_max = None

test_size = 0.5
random_state = 1

for modelattr in (ModelAttr(x, y) for x in model_names for y in postfixes):
    print("===== {}_{} =====".format(modelattr.model_name, modelattr.postfix))
    in_dist_file="softmax_scores/Base/{}_{}_Base_In.txt".format(modelattr.model_name, modelattr.postfix)
    out_dist_file="softmax_scores/Base/{}_{}_Base_Out.txt".format(modelattr.model_name, modelattr.postfix)
    scores_in = np.loadtxt(in_dist_file)
    scores_out = np.loadtxt(out_dist_file)
    scores_in_train, scores_in_test = train_test_split(scores_in, shuffle=True, test_size=test_size, random_state=random_state)
    scores_out_train, scores_out_test = train_test_split(scores_out, shuffle=True, test_size=test_size, random_state=random_state)
    
    # Compute FPR at TPR 95% using train subset only
    fpr, delta = find_best_delta_at_tpr95(scores_in=scores_in_train, scores_out=scores_out_train,
                                          delta_start=None, delta_end=None)
    print("Training FPR:{}, Delta:{}".format(fpr, delta))
    
    if fpr < fpr_min:
        fpr_min = fpr
        delta_fpr_min = delta
        modelattr_fpr_min = modelattr
        
    # Compute FPR on the test subset based on the delta found on training subset
    tpr, fpr = get_tpr_and_fpr(scores_in_test, scores_out_test, delta)
    print("Testing FPR:{}, TPR:{}".format(fpr, tpr))

    # Compute AUROC
    auroc_value = auroc(in_dist_file=in_dist_file, out_dist_file=out_dist_file)
    print("AUROC:{}\n".format(auroc_value))
    if auroc_value > auroc_max:
        auroc_max = auroc_value
        odinparam_auroc_max = modelattr

print('***** Training Min FPR at TPR 95% *****')
print("Model:{}_{}, FPR_Min:{}, Delta:{}\n"
      .format(modelattr_fpr_min.model_name, modelattr_fpr_min.postfix, fpr_min, delta_fpr_min))

print('***** Baseline Max AUROC *****')
print("Model:{}_{}, AUROC:{}"
      .format(modelattr_fpr_min.model_name, modelattr_fpr_min.postfix, auroc_max))

#### ODIN Softmax Scores

In [ ]:
from odin import find_best_delta_at_tpr95, auroc
from typing import NamedTuple
import numpy as np
import sys

OdinParam = NamedTuple('OdinParam', [('temperature', int), ('magnitude', float)])
model_name = 'DenseNet201'
postfix = 'best_balanced_acc'
temperatures = [1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]
magnitudes = np.round(np.arange(0, 0.0041, 0.0002), 4)

fpr_min = sys.float_info.max
delta_fpr_min = None
odinparam_fpr_min = None

auroc_max = sys.float_info.min
odinparam_auroc_max = None

test_size = 0.5
random_state = 1

for odinparam in (OdinParam(x, y) for x in temperatures for y in magnitudes):
    print("===== {}_{}, Temperature: {}, Magnitude: {} ====="
          .format(model_name, postfix, odinparam.temperature, odinparam.magnitude))
    in_dist_file="softmax_scores/{}_{}/{}_{}_ODIN_In.txt".format(
        odinparam.temperature, odinparam.magnitude, model_name, postfix)
    out_dist_file="softmax_scores/{}_{}/{}_{}_ODIN_Out.txt".format(
        odinparam.temperature, odinparam.magnitude, model_name, postfix)
    scores_in = np.loadtxt(in_dist_file)
    scores_out = np.loadtxt(out_dist_file)
    scores_in_train, scores_in_test = train_test_split(scores_in, shuffle=True, test_size=test_size,
                                                       random_state=random_state)
    scores_out_train, scores_out_test = train_test_split(scores_out, shuffle=True, test_size=test_size,
                                                         random_state=random_state)

    # Compute FPR at TPR 95% using training subset only
    fpr, delta = find_best_delta_at_tpr95(scores_in=scores_in_train, scores_out=scores_out_train,
                                          delta_start=None, delta_end=None)
    print("Training FPR:{}, Delta:{}".format(fpr, delta))
    if fpr < fpr_min:
        fpr_min = fpr
        delta_fpr_min = delta
        odinparam_fpr_min = odinparam
        
    # Compute FPR on the test subset based on the delta found on training subset
    tpr, fpr = get_tpr_and_fpr(scores_in_test, scores_out_test, delta)
    print("Testing FPR:{}, TPR:{}".format(fpr, tpr))

    # Compute AUROC
    auroc_value = auroc(in_dist_file=in_dist_file, out_dist_file=out_dist_file)
    print("AUROC:{}\n".format(auroc_value))
    if auroc_value > auroc_max:
        auroc_max = auroc_value
        odinparam_auroc_max = odinparam

print('***** Training Min FPR at TPR 95% *****')
print("Model:{}_{}, FPR_Min:{}, Delta:{}, Temperature: {}, Magnitude: {}\n"
      .format(model_name, postfix, fpr_min, delta_fpr_min, odinparam_fpr_min.temperature, odinparam_fpr_min.magnitude))

print('***** ODIN Max AUROC *****')
print("Model:{}_{}, AUROC:{}, Temperature: {}, Magnitude: {}"
      .format(model_name, postfix, auroc_max, odinparam_auroc_max.temperature, odinparam_auroc_max.magnitude))

#### ROC Curves of Baseline and ODIN

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

# Optimal ODIN Parameters
optimal_temperature = 2
optimal_magnitude = 0.0002
optimal_delta = 0.90385

in_dist_files = ['softmax_scores/Base/DenseNet201_best_balanced_acc_Base_In.txt',
                 "softmax_scores/{}_{}/DenseNet201_best_balanced_acc_ODIN_In.txt".format(optimal_temperature, optimal_magnitude)]
out_dist_files = ['softmax_scores/Base/DenseNet201_best_balanced_acc_Base_Out.txt',
                  "softmax_scores/{}_{}/DenseNet201_best_balanced_acc_ODIN_Out.txt".format(optimal_temperature, optimal_magnitude)]
labels = ['Baseline', 'ODIN']
colors = ['red', 'blue']

fig = plt.figure(figsize=(8, 8))
fig.patch.set_facecolor('white')
lw = 2

for i in range(len(labels)):
    scores_in = np.loadtxt(in_dist_files[i])
    scores_out = np.loadtxt(out_dist_files[i])
    y_true = np.concatenate([np.repeat(1, scores_in.size), np.repeat(0, scores_out.size)])
    y_score = np.concatenate([scores_in, scores_out])
    fpr, tpr, thresholds = roc_curve(y_true, y_score, pos_label=1)
    roc_auc = auc(fpr, tpr)
    label = "{} (area = {:.2f})".format(labels[i], roc_auc)
    plt.plot(fpr, tpr, color=colors[i], lw=lw, label=label)
    
plt.plot([0, 1], [0, 1], color='green', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate on Out-of-Distribution Set')
plt.ylabel('True Positive Rate on Validation Set')
# plt.title(title)
plt.legend(loc="lower right")
plt.show()

## Validation Set + Out-of-Distribution Set

### Step Function vs. Logistic Function for Out-of-Distribution Score

In [ ]:
from sklearn.metrics import balanced_accuracy_score
from visuals import plot_confusion_matrix
from utils import logistic

# In-distribution
df_in = pd.read_csv(os.path.join(pred_result_folder, 'Ensemble_best_balanced_acc.csv')).drop(columns=['pred_category'])
softmax_scores_in = np.loadtxt("softmax_scores/{}_{}/DenseNet201_best_balanced_acc_ODIN_In.txt"
                               .format(optimal_temperature, optimal_magnitude))
df_in['softmax_score'] = softmax_scores_in

# Out-distribution
df_out = pd.read_csv(os.path.join(out_dist_pred_result_folder, 'Ensemble_best_balanced_acc.csv')).drop(columns=['pred_category'])
df_out['category'] = category_to_index[unknown_category_name]
softmax_scores_out = np.loadtxt("softmax_scores/{}_{}/DenseNet201_best_balanced_acc_ODIN_Out.txt"
                                .format(optimal_temperature, optimal_magnitude))
df_out['softmax_score'] = softmax_scores_out

# Concatenate in- and out-distribution data
df = pd.concat([df_in, df_out])

# Step function
df.insert(loc=9, column=unknown_category_name, value=np.where(df['softmax_score'] > optimal_delta, 0.0, 1.0))
df['pred_category'] = np.argmax(np.array(df.iloc[:,1:1+len(all_category_names)]), axis=1)
print("balanced_accuracy_score:{}".format(balanced_accuracy_score(df['category'], df['pred_category'])))
# Confusion Matrix
fig = plot_confusion_matrix(df['category'], df['pred_category'], all_category_names, normalize=True,
                            title='Confusion Matrix (Step Function)',
                            figsize=(8, 6))

### Search for the best logistic growth rate or steepness of the curve

In [ ]:
import sys

best_balanced_accuracy_score = sys.float_info.min
best_k = None

for k in range(1, 1001):
    df[unknown_category_name] = 1-logistic(x=df['softmax_score'], x0=optimal_delta, k=k)
    # df[unknown_category_name] = [1-logistic(x=x, x0=optimal_delta, k=20) if x > optimal_delta else 1-logistic(x=x, x0=optimal_delta, k=5) for x in df['softmax_score']]
    df['pred_category'] = np.argmax(np.array(df.iloc[:,1:1+len(all_category_names)]), axis=1)

    # Compute Balanced Accuracy
    bac = balanced_accuracy_score(df['category'], df['pred_category'])
    if bac > best_balanced_accuracy_score:
        best_balanced_accuracy_score = bac
        best_k = k

print("best balanced_accuracy_score:{}, k:{}".format(best_balanced_accuracy_score, best_k))

df[unknown_category_name] = 1-logistic(x=df['softmax_score'], x0=optimal_delta, k=best_k)
df['pred_category'] = np.argmax(np.array(df.iloc[:,1:1+len(all_category_names)]), axis=1)

# Confusion Matrix
fig = plot_confusion_matrix(df['category'], df['pred_category'], all_category_names, normalize=True,
                            title='Confusion Matrix (Logistic Function)',
                            figsize=(8, 6))
    
# unknown = df['category'] != 8
# df[unknown]
# pred_out = df['pred_category'] == 8
# df[in_dist & pred_out]

## Test Data

### Predict Test Data by Different Models

In [ ]:
!python3 main.py /home --approach 1 --predtest --predresultfolder test_predict_results --model DenseNet201 Xception ResNeXt50